##### Copyright 2023 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# PaLM Colab Magic

This notebook introduces Colab magic commands for PaLM. Magics make it easy to develop, test, compare, and evaluate prompts from within a Colab notebook.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/tools/notebook_magic.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/tools/notebook_magic.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Setup

Follow the steps below to install and test the magics.

### Installing the PaLM magic
To use the PaLM magic commands in Colab or other IPython environment, you will first need to download and install the [`google-generativeai`](https://pypi.org/project/google-generativeai) Python package.

In [ ]:
%pip install -q google-generativeai

### Loading the PaLM magic

Next, load the `%%palm` magic by using the `%load_ext` magic:

In [ ]:
%load_ext google.generativeai.notebook

### Test the installation
To test for correct installation of the magic commands, run `%%palm --help`. Note that you will also need a PaLM API key, if you don't have one already (see next step).

In [ ]:
%%palm --help

usage: palm [-h] {run,compile,compare,eval} ...

A system for interacting with LLMs.

positional arguments:
  {run,compile,compare,eval}

options:
  -h, --help            show this help message and exit



### Getting a PaLM API key

To use the PaLM API, you will need to [create an API key](https://developers.generativeai.google/tutorials/setup). (You only need to do this step once.)

### Set the API key in the notebook

Set your API key by running the cell below.

Caution: The API key is a secret that grants access to the API from your account so make sure to remove it from the notebook before saving, and do not share any notebooks that contain API keys.

In [ ]:
%env GOOGLE_API_KEY=YOUR PALM KEY

## PaLM magic commands: `run`, `compile`, `compare`, and `evaluate`

PaLM magics provide four different commands:
1. `run`
1. `compile`
1. `compare`
1. `evaluate`

### Command: `palm run`

The `run` command sends the contents of the cell to the model.

Because running prompts is so common, PaLM magics defaults to the `run` command if no command is given. For example, the next two cells are identical.

In [ ]:
%%palm run
The opposite of hot is

,Prompt Num,Input Num,Result Num,Prompt,text_result
0,0,0,0,My favourite color is,My favorite color is green. I love the way it ...


In [ ]:
%%palm
The opposite of hot is

,Prompt Num,Input Num,Result Num,Prompt,text_result
0,0,0,0,My favourite color is,My favorite color is green. I love the way it ...


#### Understanding the output

The `Prompt` column shows the text that was sent to the model, and the `text_result` column shows the result. The other columns will be introduced as you progress through this guide.

### Prompt templates

Prompts do not have to be fixed strings. You can inject values into a prompt using template placeholders by using `{curly braces}`.

In [ ]:
english_words = {
    # Each value here (hot, cold) will be substituted in for {word} in the prompt
    'word': ['hot', 'cold']
}

In [ ]:
%%palm --inputs english_words
The opposite of {word} is

,Prompt Num,Input Num,Result Num,Prompt,text_result
0,0,0,0,English: Hello\nFrench: Bonjour\nEnglish: dog\...,chien\nEnglish: cat\nFrench: chat\nEnglish: h...
1,0,1,0,English: Hello\nFrench: Bonjour\nEnglish: cat\...,chat\nEnglish: dog\nFrench: chien\nEnglish: mo...


#### Understanding the output

The `Input Num` column tracks the index of the input word in the list(s). In
these examples, `Input Num` of `0` is `'hot'`, and `1` is `'cold'`.

#### Specifying multiple sets of inputs

You can also specify multiple sets of inputs at one time.

In [ ]:
extreme_temperatures = {
    'word': ['hot', 'cold']
}
minor_temperatures = {
    'word': ['warm', 'chilly']
}

,Prompt Num,Input Num,Result Num,Prompt,text_result
0,0,0,0,The opposite of hot is,cold.
1,0,1,0,The opposite of cold is,hot.
2,0,2,0,The opposite of warm is,cold.
3,0,3,0,The opposite of chilly is,"warm.\n\nChilly means ""slightly cold"". Warm me..."


In [ ]:
%%palm --inputs extreme_temperatures minor_temperatures
The opposite of {word} is

,Prompt Num,Input Num,Result Num,Prompt,text_result
0,0,0,0,English: Hello\nFrench: Bonjour\nEnglish: dog\...,chien\nEnglish: cat\nFrench: chat
1,0,1,0,English: Hello\nFrench: Bonjour\nEnglish: cat\...,chat
2,0,2,0,English: Hello\nFrench: Bonjour\nEnglish: cow\...,vache\nEnglish: I love you.\nFrench: Je t'aime...
3,0,3,0,English: Hello\nFrench: Bonjour\nEnglish: goat...,chevreau


### Reading data from Google Sheets

The PaLM magic can also read and write to Google Sheets. You will need to be logged in to access Sheets data. This section focuses on reading data from Sheets; a [later section](#sheets_output) shows how you can write output to a Google Sheet.

#### Log in and authorize access to Sheets

In [ ]:
#@title
from google.colab import auth
auth.authenticate_user()

import google.auth
creds, _ = google.auth.default()

from google.generativeai.notebook import magics
magics.authorize(creds)

#### Formatting a spreadsheet for use with the PaLM magic

Pass the ID or URL of a Google Sheet to the `--sheets_input_names` flag to load it up as template data.

Use the following format in your spreadsheet to use the data in a prompt template:
1. Put the names of the variables (of your prompt template) in the first row of the sheet.
1. Put the data to substitute for each variable in the rows below.

For example, if your prompt template has two variables to substitute, `name` and `temperament`, you would write your spreadsheet like this:

|name|temperament|
-----|-----------
|Milo|cheeky|
|Bigsly|relaxed|
|Subra|shy|


In [ ]:
%%palm --sheets_input_names https://docs.google.com/spreadsheets/d/1UHfpkmBqIX5RjeJcGXOevIEhMmEoKlf5f9teqwQyHqc/edit
Create a single sentence description of a monkey's personality. The monkey's name is {name} and it has a {temperament} temperament.

,Prompt Num,Input Num,Result Num,Prompt,text_result
0,0,0,0,Create a single sentence description of a monk...,Milo the monkey is always getting into trouble...
1,0,1,0,Create a single sentence description of a monk...,"Bigsly, the monkey, is relaxed and easy-going."
2,0,2,0,Create a single sentence description of a monk...,Subra is a shy monkey.


#### Try it yourself!

To try this out using your own data, create a [new Sheet](http://sheet.new/) and pass the ID to `--sheets_input_names`. As well as ID and URL, you can also search your sheets by title, e.g. `%%palm --sheets_input_names "Animal adjectives"`.

#### Combining Sheets inputs with Python inputs

Sheets inputs can also be combined with `--inputs`:

In [ ]:
new_monkeys = {
    'name': ['Hackerella'],
    'temperament': ['clever'],
}

In [ ]:
%%palm --inputs new_monkeys --sheets_input_names 1UHfpkmBqIX5RjeJcGXOevIEhMmEoKlf5f9teqwQyHqc 1UHfpkmBqIX5RjeJcGXOevIEhMmEoKlf5f9teqwQyHqc
Create a single sentence description of a monkey's personality. The monkey's name is {name} and it has a {temperament} temperament.

,Prompt Num,Input Num,Result Num,Prompt,text_result
0,0,0,0,Create a single sentence description of a monk...,Hackerella is a curious and intelligent monkey...
1,0,1,0,Create a single sentence description of a monk...,Milo is a mischievous little monkey who loves ...
2,0,2,0,Create a single sentence description of a monk...,"Bigsly the monkey is a laid-back, easygoing soul."
3,0,3,0,Create a single sentence description of a monk...,Subra is a shy monkey who enjoys being around ...
4,0,4,0,Create a single sentence description of a monk...,Milo the monkey has a cheeky temperament. He i...
5,0,5,0,Create a single sentence description of a monk...,"Bigsly the monkey is a relaxed, laid-back indi..."
6,0,6,0,Create a single sentence description of a monk...,Subra is a shy monkey. He is very timid around...


### Command: `palm eval`

Use `%%palm eval` to compare the output of a prompt with known ground-truth data.

In [ ]:
test_data = {
    "word": ["dog", "cat", "house"]
}
ground_truth = ["chien", "chat", "maison"]

In [ ]:
%%palm eval --inputs test_data --ground_truth ground_truth
English: Hello
French: Bonjour
English: {word}
French:

,Prompt Num,Input Num,Result Num,Prompt vars,actual_text_result,ground_truth_text_result,is_equal
0,0,0,0,{'word': 'dog'},chien,chien,True
1,0,1,0,{'word': 'cat'},chat,chat,True
2,0,2,0,{'word': 'house'},maison,maison,True


#### Post processing model outputs

To perform ground-truth testing, you may need to post-process the model output.

[Post-processing](#post_processing) functions allow you to define a function that processes the model output. In the case of the `eval` command, only the result column is used in the final equality check.

Use the `post_process_replace_fn` decorator to define a function to post-process results:

In [ ]:
from google.generativeai.notebook import magics

# Define a function to extract only the first response.
@magics.post_process_replace_fn
def extract_and_normalize(input):
  first_line, *unused = input.split('English:')
  return first_line.strip().lower()

The `extract_and_normalize` function defined above will take the output from the model and trim any repeated language pairs, leaving just the first response. Check out the [post-processing](#post_processing) section to learn more about post-processing.

In [ ]:
%%palm eval --inputs test_data --ground_truth ground_truth | extract_and_normalize
English: Hello
French: Bonjour
English: {word}
French:

,Prompt Num,Input Num,Result Num,Prompt vars,actual_text_result,ground_truth_text_result,is_equal
0,0,0,0,{'word': 'dog'},chien,chien,True
1,0,1,0,{'word': 'cat'},chat,chat,True
2,0,2,0,{'word': 'house'},maison,maison,True


### Command: `palm compile`

Use the `%%palm compile` command to convert a prompt with placeholders to a  function callable from within Python.

All flags and post-processing are "compiled" into the function and will be used when invoked.

In this example, a function called `translate_en_to_fr` is created, using the `extract_and_normalize` post-processing function from [before](#palm_eval).

In [ ]:
%%palm compile translate_en_to_fr | extract_and_normalize
English: Hello
French: Bonjour
English: {word}
French:

'Saved function to Python variable: translate_en_to_fr'

In [ ]:
en_words = ['cat', 'dog']
translate_en_to_fr({'word': en_words})

,Prompt Num,Input Num,Result Num,Prompt,text_result
0,0,0,0,English: Hello\nFrench: Bonjour\nEnglish: cat\...,chat
1,0,1,0,English: Hello\nFrench: Bonjour\nEnglish: dog\...,chien


#### Output formats

By default, a "compiled" function returns its output as an object that will be displayed as Pandas `DataFrame`. However, you can convert the results object to a `DataFrame` or dictionary with `.as_dict()` or `.as_dataframe()`, respectively.

For more information, see the [`--outputs`](#python_output) flag.

In [ ]:
results = translate_en_to_fr({'word': en_words}).as_dict()

fr_words = results['text_result']

for en, fr in zip(en_words, fr_words):
  print(f'{fr} is French for {en}')


chat is French for cat
chien is French for dog


### Command: `palm compare`

`%%palm compare` runs compiled prompts and produces a table with the comparison results side-by-side, so you can inspect the differences.

In [ ]:
%%palm compile few_shot_prompt
English: Hello
French: Bonjour
English: computer
French: ordinateur
English: {word}
French:

'Saved function to Python variable: few_shot_prompt'

In [ ]:
%%palm compile zero_shot_prompt
{word} translated to French is:

'Saved function to Python variable: zero_shot_prompt'

In [ ]:
words = {
    "word": ["dog", "cat", "house"]
}

In [ ]:
%%palm compare few_shot_prompt zero_shot_prompt --inputs words

,Prompt Num,Input Num,Result Num,Prompt vars,few_shot_prompt_text_result,zero_shot_prompt_text_result,is_equal
0,0,0,0,{'word': 'dog'},chien,'Dog' is 'chien' in French.,False
1,0,1,0,{'word': 'cat'},chat,'Chat' is the French word for 'cat'.,False
2,0,2,0,{'word': 'house'},maison,1. house\nHouse is a building in which a famil...,False


#### Custom comparison functions

By default, `compare` just checks for equalilty in the returned results. However, you can specify one or more custom functions with the `--compare_fn` flag:.

In [ ]:
def average_word_length(lhs, rhs):
  """Count the average number of words used across prompts."""
  return (len(lhs.split(' ')) + len(rhs.split(' '))) / 2

def shortest_answer(lhs, rhs):
  """Label the prompt that generated the shortest output."""
  if len(lhs) < len(rhs):
    return 'first'
  elif len(lhs) > len(rhs):
    return 'second'
  else:
    return 'same'

In [ ]:
%%palm compare few_shot_prompt zero_shot_prompt --inputs words --compare_fn average_word_length shortest_answer

,Prompt Num,Input Num,Result Num,Prompt vars,few_shot_prompt_text_result,zero_shot_prompt_text_result,average_word_length,shortest_answer
0,0,0,0,{'word': 'dog'},chien,1. dog (animal) -> chien\n2. dog (person) -> c...,9.0,first
1,0,1,0,{'word': 'cat'},chat,"The word ""cat"" in English can be translated to...",6.5,first
2,0,2,0,{'word': 'house'},maison,1. house\n2. maison\n\nThe French translation ...,5.0,first


## Other commands

### Help

The `--help` flag displays the supported commands that you can pass directly to `%%palm`

Append `--help` to view detailed documentation for each command. For example,

In [ ]:
%%palm run --help

usage: palm run [-h] [--model_type {echo,text}] [--temperature TEMPERATURE]
                [--model MODEL] [--candidate_count CANDIDATE_COUNT] [--unique]
                [--inputs INPUTS [INPUTS ...]]
                [--sheets_input_names SHEETS_INPUT_NAMES [SHEETS_INPUT_NAMES ...]]
                [--outputs OUTPUTS [OUTPUTS ...]]
                [--google-generativeaisheets_output_names SHEETS_OUTPUT_NAMES [SHEETS_OUTPUT_NAMES ...]]

options:
  -h, --help            show this help message and exit
  --model_type {echo,text}, -mt {echo,text}
                        The type of model to use.
  --temperature TEMPERATURE, -t TEMPERATURE
                        Controls the randomness of the output. Must be
                        positive. Typical values are in the range: [0.0, 1.0].
                        Higher values produce a more random and varied
                        response. A temperature of zero will be deterministic.
  --model MODEL, -m MODEL
                        The na

### Models

Use the `--model` flag to specify the PaLM model variant you wish to use.

See the [`list_models()`](/api/python/google/generativeai/list_models) method to retrieve the supported models. The PaLM magic can be used with any model supporting the `generateText` method.

In [ ]:
%%palm run --model models/text-bison-001
My favourite color is

,Prompt Num,Input Num,Result Num,Prompt,text_result
0,0,0,0,My favourite color is,My favorite color is blue.


#### Model parameters

You can also configure model parameters, such as [`--candidate_count`](#candidate_count) and `--temperature`.

In [ ]:
%%palm run --model models/text-bison-001 --temperature 0.5
My favourite color is

,Prompt Num,Input Num,Result Num,Prompt,text_result
0,0,0,0,My favourite color is,My favorite color is green. I love the way it ...


### Debugging: The echo model

An `echo` model is also available that will echo the prompt back to you. It does not make any API calls or consume your quota so it can be a fast and simple way to test output or post-processing.

In [ ]:
%%palm --model_type echo
A duck's quack does not echo.

,Prompt Num,Input Num,Result Num,Prompt,text_result
0,0,0,0,A duck's quack does not echo.,A duck's quack does not echo.


### Export output to Python {:#python_output}

In addition to displaying tabular output, the PaLM magic can save model output to Python variables, allowing you to manipulate them further or to export your results.

In this example, the output is saved to a Python variable: `fave_colors`


In [ ]:
%%palm --outputs fave_colors
The best colors to wear in spring-time are

,Prompt Num,Input Num,Result Num,Prompt,text_result
0,0,0,0,The best colors to wear in spring-time are,The best colors to wear in spring-time are:\n\...


Output variables are custom objects that will display as Pandas `DataFrame`s by default. They can be coerced into a Python dictionary or dataframe explicitly by calling `as_dict()` or `as_pandas_dataframe()`.

In [ ]:
from pprint import pprint

pprint(fave_colors.as_dict())

{'Input Num': [0],
 'Prompt': ['The best colors to wear in spring-time are'],
 'Prompt Num': [0],
 'Result Num': [0],
 'text_result': ['The best colors to wear in spring-time are:\n'
                 '\n'
                 '- Pastels: pastels are light and airy, and they perfectly '
                 'capture the essence of spring.\n'
                 '- Brights: bright colors are also a great choice for spring, '
                 'as they add a touch of fun and excitement to your outfit.\n'
                 '- Neutrals: neutrals are always a safe bet, and they can be '
                 'easily dressed up or down depending on the occasion.\n'
                 '\n'
                 'No matter what your personal style, there are plenty of '
                 'spring colors to choose from that will help you look your '
                 'best.']}


### Write to Google Sheets {:#sheets_output}

You can save output back to Google Sheets, using `--sheets_output_names`. You must be logged in, and you must have the appropriate permissions to access private Sheets.

To try this out, create a [new Sheet](http://sheet.new/) and name it `Translation results`. Like the input flag, the `--sheets_output_names` flag also accepts the sheet URL or ID in place of the textual name.

In [ ]:
%%palm --inputs english_words --sheets_output_names "Translation results"
English: Hello
French: Bonjour
English: {word}
French:

,Prompt Num,Input Num,Result Num,Prompt,text_result
0,0,0,0,English: Hello\nFrench: Bonjour\nEnglish: dog\...,chien\nEnglish: cat\nFrench: chat
1,0,1,0,English: Hello\nFrench: Bonjour\nEnglish: cat\...,chat English: how many ?\nFrench: Combien ?


The results are saved to a new tab and contain the same data you see here in Colab.

![Example of a saved sheet](https://developers.generativeai.google/tools/sheets_output.png)

### Generating multiple candidates {:#candidate_count}

To generate more than one output for a single prompt, you can pass `--candidate_count` to the model. This is set to 1 by default, which outputs only the top result.

Sometimes the model will generate the same output across candidates. These can be filtered with the `--unique` flag, which de-duplicates results out of the candidate batch (but not across multiple prompts).

In [ ]:
%%palm run --temperature 1.0 --candidate_count 8 --unique
In a single word, my favourite color is

,Prompt Num,Input Num,Result Num,Prompt,text_result
0,0,0,0,"In a single word, my favourite color is",cerulean
1,0,0,1,"In a single word, my favourite color is",Magenta.
2,0,0,2,"In a single word, my favourite color is",**blue**.
3,0,0,6,"In a single word, my favourite color is",magenta.


The `Result Num` column distinguishes multiple candidates generated from the same prompt.

### Post-processing model output {:#post_processing}

The broad range of possible outputs and structures can make it difficult to adapt the model's output to your problem domain. The PaLM magic provides post-processing options that allow you to modify or process model output using Python code.

Post-processing functions can either add a new column to the output, or modify the `text_result` column. The `text_result` column is the last column, and is used by the `eval` and `compare` commands to determine the final output.

Here are some sample functions to use in post-processing. One adds a new column and the other updates the result column, using the `post_process_replace_fn` decorator.

In [ ]:
import re
from google.generativeai.notebook import magics

# Add a new column.
def word_count(result):
  return len(result.split(' '))

# Modify the text_result column
@magics.post_process_replace_fn
def extract_first_sentence(result):
  """Extracts the first word from the raw result."""
  first, *_ = re.split(r'\.\s*', result)
  return first

To use these functions, append them to the `%%palm` command using the pipe (`|`) operator, like so.

In [ ]:
%%palm run | word_count | extract_first_sentence
The happiest thing I can imagine is

,Prompt Num,Input Num,Result Num,Prompt,word_count,text_result
0,0,0,0,The happiest thing I can imagine is,64,The happiest thing I can imagine is cuddling u...


Order matters here. When `word_count` is invoked, the original model output is used to calculate the number of words. If you swap these around, the word count would instead be the number of words in the extracted first sentence.

## Further reading

* Refer to the [LLMs concepts guide](https://developers.generativeai.google/guide/concepts) to learn more about LLMs.
* Check out the [prompt guidelines](https://developers.generativeai.google/guide/prompt_best_practices) to learn more about crafting prompts to get the most out of working with PaLM.
* To prototype and experiment with different prompts, check out [MakerSuite](https://makersuite.google.com/). Also, refer to the [MakerSuite quickstart](https://developers.generativeai.google/tutorials/makersuite_quickstart) for more information.